In [2]:
#19101185 강동현 데이터분석 Assignment 07 Textbook - Applied
#Problem 5

#validation set approach를 이용하여 test error of this logistic regression model를 예측한다.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

Default = pd.read_csv('C:\\Users\\car99\\DATA7\\Default.csv', index_col=0)

In [5]:
#a번 - income과 balance를 이용하여 logistic regression model을 fit 하여라.

x01 = sm.add_constant(Default.iloc[:, 2:5]) 
y01 = np.where(Default['default']=='No', 0, 1) 

glm1 = sm.Logit(y01, x01)
print(glm1.fit().summary())

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 03 May 2021   Pseudo R-squ.:                  0.4594
Time:                        02:04:12   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0

In [6]:
#b번 - validation set approach을 이용하여 test error를  추정한다.
#과정은 다음과 같다.

#i - sample을 training set 과 validation set 으로 나눈다.
x = pd.DataFrame(Default.iloc[:, 2:5])
y = np.array(Default['default'])
np.random.seed(1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

#ii - training만 가지고 multiple logistic regression model을 fitting 한다.
glm2 = LogisticRegression()

#iii - validation set안의 개인의 확률을 확인 하고 0.5를 기준으로 카테고리를 분류한다.
glm2.pred = glm2.fit(x_train, y_train).predict(x_test)

#iv - validation set error를 계산한다.
error1 = 1-accuracy_score(y_test, glm2.pred)
print(error1)

0.030750000000000055


In [7]:
#c번 - b번의 과정을 3번 반복하는데 이때, 각기 다른 splits 즉, 이전의 내용과
#다른 내용의 training set 과 validation set을 통해 b번의 과정을 3번 반복하는 문제이다.

x = pd.DataFrame(Default.iloc[:, 3:5])
y = np.array(Default['default'])

error2 = np.zeros(3)
for i in range(2, 5, 1):
    np.random.seed(i)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    
    glm3 = LogisticRegression()
    glm3.pred = glm2.fit(x_train, y_train).predict(x_test)
    
    error3 = 1-accuracy_score(y_test, glm3.pred)
    print(error3)
    error2[i-2:i-1] = error3

print("3 times result:", end=" ")
print(error2)

0.03025
0.03374999999999995
0.03300000000000003
3 times result: [0.03025 0.03375 0.033  ]


In [8]:
#d번 - income, balance, and a dummy variable for student를 사용하여 
#logistic regression model의 디폴트 확률을 예측해라. 또한 test error for this model을 validation set approach
#를 이용하여 추정하라. 또한 dummy variable for student가 test error rate의 감소를 이끄는지 확인하는 문제이다.

#정리하자면 우리는 dummy variable for student을 추가한 후 test error rate를 확인하여 이전에 구한 것들과 비교해
#dummy variable for student의 추가가 test error rate의 감소에 영향이 있거나 이끌었는지를 확인하면 된다.
#이 과정에서 logistic regression model의 디폴트 확률을 예측하고 또한 test error rate는
#validation set approach를 통하여 살펴볼 것이다.

Default['student_Yes'] = np.where(Default['student']=="Yes",1,0)
x = Default[['student_Yes','income','balance']]
y = Default['default']

np.random.seed(10)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

glm4 = LogisticRegression()

#iii - validation set안의 개인의 확률을 확인 하고 0.5를 기준으로 카테고리를 분류한다.
glm4.pred = glm4.fit(x_train, y_train).predict(x_test)

#iv - validation set error를 계산한다.
error4 = 1-accuracy_score(y_test, glm4.pred)
print(error4)

#dummy variable을 추가하였음에도 test error rate에는 큰 변화가 없다. 따라서 dummy variable for student는
#test error rate에 큰 영향을 미치지 못하고 있다.

0.03325
